In [1]:
import price
import pandas as pd
from fmp_db import fmp
pd.options.display.float_format = '{:.3f}'.format

In [2]:
db = fmp()

In [4]:
df = db.load_financials()
df['marketCap'] = df['marketCap']/100000000
df

,symbol,name,exchange,sector,industry,marketCap,numEmployees,revenue_per_share_fiveY_growth,gross_profit_margin,roe,eps_growth,div_yield,div_per_share,dps_fiveY_growth
0,A,Agilent Technologies Inc,NYSE,Healthcare,Diagnostics & Research,406.132,16400,0.619,0.534,0.167,0.306,0.006,0.748,0.669
1,AAL,American Airlines Group Inc,NASDAQ,Industrials,Airlines,155.471,113200,-0.594,-0.762,1.343,0.540,-1.000,-1.000,-1.000
2,AAP,Advance Auto Parts Inc,NYSE,Consumer Cyclical,Specialty Retail,123.820,40000,4.293,0.442,0.240,2.343,0.007,1.250,0.000
3,AAPL,Apple Inc,NASDAQ,Technology,Consumer Electronics,20794.400,100000,1.333,0.399,1.118,-0.171,0.007,0.835,0.564
4,ABBV,Abbvie Inc,NYSE,Healthcare,Drug Manufacturers—General,2025.850,48000,0.996,0.648,0.364,97.072,0.043,4.960,1.298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,YUM,Yum! Brands Inc,NYSE,Consumer Cyclical,Restaurants,357.392,38000,-1.922,0.331,-0.112,0.166,0.016,1.940,0.031
500,ZBH,Zimmer Biomet Holdings Inc,NYSE,Healthcare,Medical Devices,350.938,20000,-0.067,0.698,0.026,1437.090,0.006,0.960,0.074
501,ZBRA,Zebra Technologies Corp,NASDAQ,Technology,Communication Equipment,265.978,8800,0.522,0.461,0.310,0.142,-1.000,-1.000,0.000
502,ZION,Zions Bancorporation NA,NASDAQ,Financial Services,Banks—Regional,92.916,9682,0.599,0.255,0.077,3.708,0.024,1.360,1.997


## Minimum Fundamental Requirements

In [5]:
minMktCap = 1000
minRevGrowth = 0.3
minGPMargin = 0.2
minEPSGrowth = 0.1
minROE = 0.2
minDPSGrowth = 0.1
minDivYield = 0

In [7]:
df = df[df['marketCap'] > minMktCap]
conditions = (df['revenue_per_share_fiveY_growth'] > minRevGrowth) & (df['gross_profit_margin'] > minGPMargin) & (df['eps_growth'] > minEPSGrowth) & (df['roe'] > minROE)
df = df[conditions]
div_conds = (df['dps_fiveY_growth'] > minDPSGrowth) & (df['div_yield'] > minDivYield)
df_final = df[div_conds]

In [ ]:
mom_data = {
    'symbol': [],
    'currentPrice': [],
    'recentHigh': [],
    '15%_discount': [],
    '20%_discount': [],
    '25%_discount': []
}

In [ ]:
for symbol in list(df_final['symbol']):
    print(symbol)
    currPrice = db.get_current_price(symbol)
    high = price.calculate_prev_max_high(symbol, 90)
    fifteen = high * 0.85
    twenty = high * 0.80
    twentyFive = high * 0.75
    
    mom_data['symbol'].append(symbol)
    mom_data['currentPrice'].append(currPrice)
    mom_data['recentHigh'].append(high)
    mom_data['15%_discount'].append(fifteen)
    mom_data['20%_discount'].append(twenty)
    mom_data['25%_discount'].append(twentyFive)

In [ ]:
mom_df = pd.DataFrame(mom_data)
mom_df.set_index('symbol', inplace=True)
mom_df

In [ ]:
buyList = []

for symbol in list(mom_df.index):
    currPrice = mom_df.loc[symbol,'currentPrice']
    if currPrice < mom_df.loc[symbol,'25%_discount']:
        print(f'{symbol} \t 25% discount!')
    elif currPrice < mom_df.loc[symbol, '20%_discount']:
        print(f'{symbol} \t 20% discount!')
    elif currPrice < mom_df.loc[symbol, '15%_discount']:
        print(f'{symbol} \t 15% discount!')
    else:
        print(f'{symbol} \t still expensive :(')